# Desafio Técnico - Cientista de Dados Júnior
## Ermeson Freitas

In [1]:
# Importanto bibliotecas
import pandas as pd
import basedosdados as bd

In [2]:
# Lendo tabela de chamados
query = """
SELECT *
FROM `datario.administracao_servicos_publicos.chamado_1746` 
WHERE data_particao = "2023-04-01" and
DATE(data_inicio) = "2023-04-01";
"""
# Salvando consulta em um dataframe
df_chamados = bd.read_sql(query, billing_project_id="desafio-emd-cdj")

# Visuliazando df
df_chamados.head(5)

Downloading: 100%|██████████| 73/73 [00:00<00:00, 147.01rows/s]


,id_chamado,data_inicio,data_fim,id_bairro,id_territorialidade,id_logradouro,numero_logradouro,id_unidade_organizacional,nome_unidade_organizacional,id_unidade_organizacional_mae,...,tempo_prazo,prazo_unidade,prazo_tipo,dentro_prazo,situacao,tipo_situacao,justificativa_status,reclamacoes,data_particao,geometry
0,18516246,2023-04-01 00:55:38,2023-04-01 00:55:38,None,None,None,NaN,1706,TR/SUBOP/CFT - Coordenadoria de Fiscalização e...,SMTR - Secretaria Municipal de Transportes,...,None,D,F,No prazo,Encerrado,Atendido parcialmente,None,0,2023-04-01,None
1,18516254,2023-04-01 01:08:04,2023-04-01 01:43:38,1,1,62364,60.0,70,GM-RIO - Guarda Municipal do Rio de Janeiro,GM-RIO - Guarda Municipal do Rio de Janeiro,...,None,H,F,No prazo,Encerrado,Não constatado,None,0,2023-04-01,None
2,18516194,2023-04-01 00:04:59,NaT,14,1,76992,120.0,1240,CCU - 1º Serviço de Fiscalização,CCU - Coordenadoria de Controle Urbano,...,None,D,F,Fora do prazo,Não Encerrado,Andamento,None,0,2023-04-01,POINT(-43.186679 -22.921335)
3,18516238,2023-04-01 00:38:22,NaT,5,1,60483,319.0,1258,CFE - Coordenação de Feiras,SEOP - Secretaria Municipal de Ordem Pública,...,None,D,F,Fora do prazo,Não Encerrado,Andamento,None,0,2023-04-01,POINT(-43.1882526 -22.9122295)
4,18516208,2023-04-01 00:14:00,NaT,14,1,76992,90.0,82,CLF - Coordenadoria de Licenciamento e Fiscali...,CLF - Coordenadoria de Licenciamento e Fiscali...,...,None,D,F,Fora do prazo,Não Encerrado,Andamento,None,0,2023-04-01,POINT(-43.1872883 -22.9211226)


### 1. Quantos chamados foram abertos no dia 01/04/2023?

In [3]:
# Realiazamos a contagem de registros no df_chamados que corresponde ao número total de chamados abertos
# no dia 01/04/2023.
len(df_chamados)

73

### 2. Qual o tipo de chamado que teve mais reclamações no dia 01/04/2023?

In [4]:
df_chamados['tipo'].value_counts().rename_axis('tipo').reset_index(name='quantidade').head(1)
# Portanto, Poluição sonora é o tipo de chamado com maior ocorrência no dia 01/04/2023.


,tipo,quantidade
0,Poluição sonora,24


### 3. Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?

In [5]:
# Lendo tabela de bairros
query = """
SELECT id_bairro, nome, subprefeitura
FROM `datario.dados_mestres.bairro`;
"""
# Salvando consulta em um dataframe
df_bairros = bd.read_sql(query, billing_project_id="desafio-emd-cdj")

# Visuliazando df
df_bairros.head(5)

Downloading: 100%|██████████| 164/164 [00:00<00:00, 389.61rows/s]


,id_bairro,nome,subprefeitura
0,2,Gamboa,Centro
1,1,Saúde,Centro
2,4,Caju,Centro
3,3,Santo Cristo,Centro
4,161,Lapa,Centro


In [6]:
# Juntando os DataFrames
df = pd.merge(df_chamados, df_bairros, on='id_bairro', how='left')

# Contando o número de chamados para cada bairro
df['nome'].value_counts().rename_axis('bairro').reset_index(name='quantidade').head(3)

,bairro,quantidade
0,Engenho de Dentro,8
1,Campo Grande,6
2,Leblon,6


RESPOSTA: Engenho de Dentro com 8 chamados, Campo Grande e Leblon com 6 chamados.

### 4. Qual o nome da subprefeitura com mais chamados abertos nesse dia?

In [7]:
df['subprefeitura'].value_counts().rename_axis('subprefeitura').reset_index(name='quantidade').head(1)

,subprefeitura,quantidade
0,Zona Norte,25


RESPOSTA: Zona Norte com 25 chamados.

### 5. Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?

In [8]:
df[(df['id_bairro'].isnull()) | (df['subprefeitura'].isnull())]

,id_chamado,data_inicio,data_fim,id_bairro,id_territorialidade,id_logradouro,numero_logradouro,id_unidade_organizacional,nome_unidade_organizacional,id_unidade_organizacional_mae,...,prazo_tipo,dentro_prazo,situacao,tipo_situacao,justificativa_status,reclamacoes,data_particao,geometry,nome,subprefeitura
0,18516246,2023-04-01 00:55:38,2023-04-01 00:55:38,None,None,None,NaN,1706,TR/SUBOP/CFT - Coordenadoria de Fiscalização e...,SMTR - Secretaria Municipal de Transportes,...,F,No prazo,Encerrado,Atendido parcialmente,None,0,2023-04-01,None,NaN,NaN


RESPOSTA: Sim, há um chamado que não foi associado a um bairro e nem a uma subprefeitura neste dia. id_chamado = 18516246, realizado pela SMTR - Secretaria Municipal de Transportes este chamado é da categoria serviço, tipo = Ônibus e subtipo = Verificação de ar condicionado inoperante no ônibus.
Isto ocorre por que a natureza do chamado (chamados internos) podem envolver atividades ou processos que não estão diretamente relacionados a uma localização geográfica específica. Por exemplo, um chamado para manutenção de equipamentos internos, treinamento de funcionários ou gerenciamento de projetos pode ser tratado internamente pelo departamento responsável e não exigir uma associação a uma localidade.

### 6. Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?

In [9]:
# Lendo tabela de chamados filtrando por subtipo e período
query = """
SELECT tb_a.id_subtipo, tb_a.data_inicio
FROM `datario.administracao_servicos_publicos.chamado_1746` tb_a
WHERE 
data_particao BETWEEN "2022-01-01" AND "2023-12-31"
AND 
DATE(data_inicio) BETWEEN "2022-01-01" AND "2023-12-31"
AND tb_a.id_subtipo = "5071";
"""
# Salvando consulta em um dataframe
df_chamados_2022_2023 = bd.read_sql(query, billing_project_id="desafio-emd-cdj")

# Visualizando df
df_chamados_2022_2023.head(5)

Downloading: 100%|██████████| 42408/42408 [00:02<00:00, 16081.61rows/s]


,id_subtipo,data_inicio
0,5071,2023-03-31 19:36:34
1,5071,2023-03-31 20:01:36
2,5071,2023-03-31 18:39:59
3,5071,2023-03-31 18:15:46
4,5071,2023-03-31 23:43:50


In [10]:
# Contagem do subtipo "Pertubação do sossego"
len(df_chamados_2022_2023)

42408

RESPOSTA: 42408 chamados.

### 7. Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).

In [11]:
# Lendo tabela de eventos
query = """
SELECT *
FROM `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`;
"""
# Salvando consulta em um dataframe
df_eventos = bd.read_sql(query, billing_project_id="desafio-emd-cdj")

Downloading: 100%|██████████| 4/4 [00:00<00:00,  9.37rows/s]


In [12]:
# Visuliazando df
df_eventos

,ano,data_inicial,data_final,evento,taxa_ocupacao
0,18/02 a 21/02 de 2023,2023-02-18,2023-02-21,Carnaval,0.9554
1,30-31/12 e 01/01 (2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251
2,02/09 a 04/09 de 2022,2022-09-02,2022-09-04,Rock in Rio,0.8184
3,08/09 a 11/09 de 2022,2022-09-08,2022-09-11,Rock in Rio,0.9451


In [13]:
# Copiando df_chamados_2022_2023
df_ch_2022_2023 = df_chamados_2022_2023.copy()

In [14]:
# Aqui tive dificuldades para converter as datas, fiz uma cópia dos dfs para preservar os originais.
# Primeiro extraimos apenas a data de df_ch_2022_2023['data_inicio'], ao converter a variável
# ficará do tipo object, depois é só converter novamente para to_datetime.
# len(df_ch_2022_2023)
df_ch_2022_2023['data_inicio'] = pd.to_datetime(df_ch_2022_2023['data_inicio']).dt.date
df_ch_2022_2023['data_inicio'] = pd.to_datetime(df_ch_2022_2023['data_inicio'])
# df_ch_2022_2023.dtypes
df_ch_2022_2023.head(5)

,id_subtipo,data_inicio
0,5071,2023-03-31
1,5071,2023-03-31
2,5071,2023-03-31
3,5071,2023-03-31
4,5071,2023-03-31


In [15]:
# Expansão do df_eventos para incluir todas as datas entre 'data_inicial' e 'data_final'
df_eventos2 = df_eventos.copy()
df_eventos2 = df_eventos2.assign(data_inicial=df_eventos2.apply(lambda r:
                                                             pd.date_range(r['data_inicial'],
                                                                           r['data_final']).tolist(),axis=1)).explode('data_inicial')

# Fazendo o merge interno de df_chamados_2022_2023 e df_eventos
df = pd.merge(df_ch_2022_2023, df_eventos2, left_on='data_inicio',
               right_on='data_inicial', how='inner')
len(df)

1212

RESPOSTA: 1212 chamados do subtipo "Perturbação do sossego" foram abertos durante o Reveillon, Carnaval e Rock in Rio.

### 8. Quantos chamados desse subtipo foram abertos em cada evento?

In [16]:
# df['evento'].value_counts().rename_axis('Eventos').reset_index(name='quantidade')
df['evento'].value_counts().rename_axis('Eventos')
# df.head(3)

Eventos
Rock in Rio    834
Carnaval       241
Reveillon      137
Name: evento, dtype: int64

RESPOSTA: Rock in Rio teve 834 chamados, Carnaval teve 241 chamados e Reveillon teve 137 chamados.

### 9. Qual evento teve a maior média diária de chamados abertos desse subtipo?

In [17]:
df.head(3)

,id_subtipo,data_inicio,ano,data_inicial,data_final,evento,taxa_ocupacao
0,5071,2022-12-30,30-31/12 e 01/01 (2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251
1,5071,2022-12-30,30-31/12 e 01/01 (2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251
2,5071,2022-12-30,30-31/12 e 01/01 (2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251


In [18]:
df.groupby(['evento', pd.Grouper(key='data_inicio', freq='D')]).size().groupby('evento').mean()

evento
Carnaval        60.250000
Reveillon       45.666667
Rock in Rio    119.142857
dtype: float64

RESPOSTA: Rock in Rio com 119 chamados em média por dia do subtipo "Perturbação do sossego".

### 10. Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo considerando todo o período de 01/01/2022 até 31/12/2023.

In [19]:
df_ch_2022_2023.head(3)

,id_subtipo,data_inicio
0,5071,2023-03-31
1,5071,2023-03-31
2,5071,2023-03-31


In [36]:
# Calculando a média diária de chamados por evento
media_eventos = df.groupby(['evento', pd.Grouper(key='data_inicio', freq='D')]).size().groupby('evento').mean()

# Calculando a média diária de chamados dos anos 2022 e 2023
media_2022_2023 = len(df_ch_2022_2023) / df_ch_2022_2023['data_inicio'].nunique()

# Criando a tabela de comparação
tabela_comparacao = pd.DataFrame({
    'Média do Evento': media_eventos,
    'Média 2022 a 2023': media_2022_2023,
    'Variação Percentual': ((media_eventos / media_2022_2023 - 1) * 100).round(2).astype(str) + '%'
})

print(tabela_comparacao)

             Média do Evento  Média 2022 a 2023 Variação Percentual
evento                                                             
Carnaval           60.250000          63.201192              -4.67%
Reveillon          45.666667          63.201192             -27.74%
Rock in Rio       119.142857          63.201192              88.51%


RESPOSTA: Durante o período de 01/01/2022 a 31/12/2023, a média diária de chamados foi de 63. Em comparação, o Rock in Rio apresentou a maior média diária com 119 chamados, um aumento de 88,5%. O Carnaval teve uma média de 60 chamados, uma redução de 4,6%, enquanto o Reveillon registrou uma média de 46 chamados, uma redução de 27,7%.